In [1]:
import numpy as np
import keras
np.random.seed(2016)

import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings
import os
import glob

warnings.filterwarnings("ignore")

from sklearn.cross_validation import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn.metrics import log_loss
from keras import __version__ as keras_version
import data_set_ooc
%load_ext autoreload
%autoreload 2



Using TensorFlow backend.
//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [123]:
FLAGS = dict()
FLAGS['width'] = 32
FLAGS['height'] = 32
FLAGS['batch_size'] = 20
FLAGS['kernel_1_out'] = 8
FLAGS['kernel_2_out'] = 8
FLAGS['conv2_input_width'] = 16
FLAGS['conv2_input_height'] = 16
FLAGS['n_classes'] = 8
FLAGS['learning_rate'] = 0.001
FLAGS['batch_size'] = 16
FLAGS['n_epochs'] = 20
FLAGS['train_report_step'] = 20
FLAGS['val_report_step'] = 80
FLAGS['keep_prob'] = 0.75
FLAGS['reg'] = 0.01
FLAGS['patience'] = 5

In [130]:
data = data_set_ooc.DataSet(width = FLAGS['width'], height=FLAGS['height'])

In [131]:
from keras.callbacks import TensorBoard
tb = TensorBoard(log_dir='keras_tb/', histogram_freq=5, write_graph=True, write_images=False)

In [132]:
def create_model():
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(32, 32, 3)))
    model.add(Convolution2D(4, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(4, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(8, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(8, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='softmax'))

    sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy')

    return model


In [133]:
model = create_model()

In [134]:
def eval_on_val(data, model, batch_size=20):
    data.reset_val_index()
    n_batches = int((data.num_val_examples / batch_size))
    val_loss = 0
    for batch in range(n_batches):
        (X_batch, y_batch) = data.next_val_batch(batch_size) 
        
        val_loss += model.test_on_batch(X_batch, y_batch)
    return (val_loss / n_batches)
 

In [ ]:
n_steps = int(FLAGS['n_epochs'] * (data.num_examples / FLAGS['batch_size']))
print('training for {} steps'.format(n_steps))
X_val, y_val = data.get_validation_set()
best_val_loss = 20
no_val_improvements = 0
for step in range(n_steps):
    X_batch, y_batch = data.next_batch(FLAGS['batch_size'])
    batch_loss = model.train_on_batch(X_batch, y_batch)
    if ((step + 1) % 20 == 0):
        print('loss for batch {}:{}'.format(step, batch_loss))
        
    if ((step + 1) % 100 == 0):
        print('Evaluating on validation set')
        val_loss = eval_on_val(data, model, FLAGS['batch_size'])
        if val_loss < best_val_loss:
            print('Validation improved')
            best_val_loss = val_loss
            no_val_improvements = 0
        else:
            no_val_improvements +=1
            print('Validation has not improved for {} rounds'.format(no_val_improvements))
            if no_val_improvements == FLAGS['patience']:
                print('Validation loss has stopped improving. Stopping training')
                break
        print('validation loss for step {}:{}'.format(step, val_loss))

training for 3776 steps
loss for batch 19:1.6890984773635864
loss for batch 39:1.8800694942474365
loss for batch 59:1.9192373752593994
loss for batch 79:1.5746066570281982
loss for batch 99:1.560589075088501
Evaluating on validation set
Validation improved
validation loss for step 99:1.6214727366224249
loss for batch 119:1.8147897720336914
loss for batch 139:1.6213645935058594
loss for batch 159:1.610617995262146
loss for batch 179:1.419062852859497
loss for batch 199:1.0366400480270386
Evaluating on validation set
Validation improved
validation loss for step 199:1.6128579885401624
loss for batch 219:1.3554584980010986
loss for batch 239:1.6205137968063354
loss for batch 259:2.0612850189208984
loss for batch 279:1.8519896268844604
loss for batch 299:1.5266534090042114
Evaluating on validation set


In [109]:
import load_images_ooc
X_test, y_test = load_images_ooc.load_test(width = FLAGS['width'], 
                                                      height=FLAGS['height'])

loading 0 of 1000
loading 20 of 1000
loading 40 of 1000
loading 60 of 1000
loading 80 of 1000
loading 100 of 1000
loading 120 of 1000
loading 140 of 1000
loading 160 of 1000
loading 180 of 1000
loading 200 of 1000
loading 220 of 1000
loading 240 of 1000
loading 260 of 1000
loading 280 of 1000
loading 300 of 1000
loading 320 of 1000
loading 340 of 1000
loading 360 of 1000
loading 380 of 1000
loading 400 of 1000
loading 420 of 1000
loading 440 of 1000
loading 460 of 1000
loading 480 of 1000
loading 500 of 1000
loading 520 of 1000
loading 540 of 1000
loading 560 of 1000
loading 580 of 1000
loading 600 of 1000
loading 620 of 1000
loading 640 of 1000
loading 660 of 1000
loading 680 of 1000
loading 700 of 1000
loading 720 of 1000
loading 740 of 1000
loading 760 of 1000
loading 780 of 1000
loading 800 of 1000
loading 820 of 1000
loading 840 of 1000
loading 860 of 1000
loading 880 of 1000
loading 900 of 1000
loading 920 of 1000
loading 940 of 1000
loading 960 of 1000
loading 980 of 1000


In [111]:
preds = model.predict(X_test)

In [113]:
import make_submission
submit = make_submission.makeSubmission(preds, y_test)

In [116]:
submit.tail(20)

,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT,image
980,0.409512,7.518994e-02,3.706961e-02,2.602655e-02,7.594233e-02,1.657486e-01,6.505529e-02,1.454561e-01,img_07663.jpg
981,0.409512,7.518994e-02,3.706961e-02,2.602655e-02,7.594233e-02,1.657486e-01,6.505529e-02,1.454561e-01,img_07678.jpg
982,0.409512,7.518994e-02,3.706961e-02,2.602655e-02,7.594233e-02,1.657486e-01,6.505529e-02,1.454561e-01,img_07689.jpg
983,0.409512,7.518994e-02,3.706961e-02,2.602655e-02,7.594233e-02,1.657486e-01,6.505529e-02,1.454561e-01,img_07700.jpg
984,1.000000,3.427635e-19,0.000000e+00,3.698772e-33,1.816197e-15,4.180287e-26,0.000000e+00,2.602469e-27,img_07717.jpg
985,0.409512,7.518994e-02,3.706961e-02,2.602655e-02,7.594233e-02,1.657486e-01,6.505529e-02,1.454561e-01,img_07746.jpg
986,0.409512,7.518994e-02,3.706961e-02,2.602655e-02,7.594233e-02,1.657486e-01,6.505529e-02,1.454561e-01,img_07757.jpg
987,0.409512,7.518994e-02,3.706961e-02,2.602655e-02,7.594233e-02,1.657486e-01,6.505529e-02,1.454561e-01,img_07792.jpg
988,0.409512,7.518994e-02,3.706961e-02,2.602655e-02,7.594233e-02,1.657486e-01,6.505529e-02,1.454561e-01,img_07799.jpg
989,0.998858,2.355931e-04,5.116723e-08,5.064218e-07,8.529955e-04,3.684033e-05,8.246242e-08,1.627450e-05,img_07818.jpg


In [117]:
submit.to_csv('keras_test_ooc_overfit.csv', index=False)

In [16]:
callbacks a= [
            EarlyStopping(monitor='val_loss', patience=5, verbose=0),
tb]
hist = model.fit(data.X_train, data.y_train, batch_size=FLAGS['batch_size'], nb_epoch=FLAGS['epochs'],
          shuffle=True, verbose=2, validation_data=(X_val, y_val),
              callbacks=callbacks)



Train on 3021 samples, validate on 756 samples
INFO:tensorflow:Summary name convolution2d_1_W:0 is illegal; using convolution2d_1_W_0 instead.
INFO:tensorflow:Summary name convolution2d_1_b:0 is illegal; using convolution2d_1_b_0 instead.
INFO:tensorflow:Summary name convolution2d_2_W:0 is illegal; using convolution2d_2_W_0 instead.
INFO:tensorflow:Summary name convolution2d_2_b:0 is illegal; using convolution2d_2_b_0 instead.
INFO:tensorflow:Summary name convolution2d_3_W:0 is illegal; using convolution2d_3_W_0 instead.
INFO:tensorflow:Summary name convolution2d_3_b:0 is illegal; using convolution2d_3_b_0 instead.
INFO:tensorflow:Summary name convolution2d_4_W:0 is illegal; using convolution2d_4_W_0 instead.
INFO:tensorflow:Summary name convolution2d_4_b:0 is illegal; using convolution2d_4_b_0 instead.
INFO:tensorflow:Summary name dense_1_W:0 is illegal; using dense_1_W_0 instead.
INFO:tensorflow:Summary name dense_1_b:0 is illegal; using dense_1_b_0 instead.
INFO:tensorflow:Summary n

loading 0 of 1000
loading 20 of 1000
loading 40 of 1000
loading 60 of 1000
loading 80 of 1000
loading 100 of 1000
loading 120 of 1000
loading 140 of 1000
loading 160 of 1000
loading 180 of 1000
loading 200 of 1000
loading 220 of 1000
loading 240 of 1000
loading 260 of 1000
loading 280 of 1000
loading 300 of 1000
loading 320 of 1000
loading 340 of 1000
loading 360 of 1000
loading 380 of 1000
loading 400 of 1000
loading 420 of 1000
loading 440 of 1000
loading 460 of 1000
loading 480 of 1000
loading 500 of 1000
loading 520 of 1000
loading 540 of 1000
loading 560 of 1000
loading 580 of 1000
loading 600 of 1000
loading 620 of 1000
loading 640 of 1000
loading 660 of 1000
loading 680 of 1000
loading 700 of 1000
loading 720 of 1000
loading 740 of 1000
loading 760 of 1000
loading 780 of 1000
loading 800 of 1000
loading 820 of 1000
loading 840 of 1000
loading 860 of 1000
loading 880 of 1000
loading 900 of 1000
loading 920 of 1000
loading 940 of 1000
loading 960 of 1000
loading 980 of 1000


In [18]:
preds = model.predict(X_test, batch_size=FLAGS['batch_size'], verbose=2)


In [24]:
import make_submission
submit = make_submission.makeSubmission(preds,y_test)

In [27]:
submit.to_csv('keras_32_32.csv', index=False)